In [12]:

"""""""""""""""""""""""""""""""""""""""
network.py
Модуль создания и обучения нейронной сети для распознавания рукописных цифр
с использованием метода градиентного спуска.
Группа: КЭ-120
ФИО: Вавилин Александр Владимирович
"""""""""""""""""""""""""""""""""""""""


import random
import numpy as np



class Network(object):
    def __init__(self, sizes):
        self.layers_num = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
        
        
    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    
    def SGD(
        self,
        training_data,
        epochs,
        mini_batch_size,
        eta,
        test_data
    ):
        test_data = list(test_data)
        test_len = len(test_data)
        training_data = list(training_data)
        n = len(training_data)
        
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
                
            print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), test_len))
            
            
    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
            self.weights = [w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
            self.biases = [b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
            
            
    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        activation = x
        activations = [x]
        zs = []
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            
            activation = sigmoid(z)
            activations.append(activation)
            
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])

        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.layers_num):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

        return (nabla_b, nabla_w)
    
    
    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
                   
                   
    def cost_derivative(self, output_activations, y):
        return (output_activations-y)
                   
                   


def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))
   
                   
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

In [13]:
# MNIST LOADER
import gzip
import pickle



def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    return (training_data, validation_data, test_data)


def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    
    return (training_data, validation_data, test_data)


def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


# MAIN
tr_d, va_d, te_d = load_data_wrapper()
net = Network([784, 30, 10])
net.SGD(tr_d, 30, 10, 3.0, test_data=te_d)

Epoch 0: 8059 / 10000
Epoch 1: 8706 / 10000
Epoch 2: 8821 / 10000
Epoch 3: 8695 / 10000
Epoch 4: 8797 / 10000
Epoch 5: 8892 / 10000
Epoch 6: 8857 / 10000
Epoch 7: 8981 / 10000
Epoch 8: 8938 / 10000
Epoch 9: 8954 / 10000
Epoch 10: 9103 / 10000
Epoch 11: 8907 / 10000


/tmp/ipykernel_7944/478921467.py:108: RuntimeWarning: overflow encountered in exp
  return 1.0/(1.0 + np.exp(-z))


Epoch 12: 8858 / 10000
Epoch 13: 8960 / 10000
Epoch 14: 8958 / 10000
Epoch 15: 9090 / 10000
Epoch 16: 9095 / 10000
Epoch 17: 9071 / 10000
Epoch 18: 9233 / 10000
Epoch 19: 9000 / 10000
Epoch 20: 9079 / 10000
Epoch 21: 9042 / 10000
Epoch 22: 9156 / 10000
Epoch 23: 9173 / 10000
Epoch 24: 8998 / 10000
Epoch 25: 9162 / 10000
Epoch 26: 9178 / 10000
Epoch 27: 9164 / 10000
Epoch 28: 9083 / 10000
Epoch 29: 9125 / 10000
